# Consignas

1)	Desarrollar al menos dos modelos de clasificación (decision tree, logistic regression) que tengan como target feature categoria_socio.
La target feature podrá asumir dos valores: 
●	Standard.
●	 Silver + Gold.
Aplicar las buenas prácticas en la construcción de modelos: dividir datos en conjunto de entrenamiento y validación, cross-validation, grid-search.
Obtener feature importance.

El objetivo es entender las diferencias entre las distintas categorías de afiliados. Cuáles son las variables más relevantes para diferenciarlos?
Las conclusiones obtenidas a partir de este modelo resultarán de gran importancia para el desarrollo de un modelo de recomendación.



In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling
import seaborn as sns
import ptitprince as pt
from sklearn.preprocessing import MinMaxScaler

In [11]:
df_procesado=pd.read_csv('../Data/df_procesado.csv')

In [13]:
df_procesado

,id_consumo_encoded,id_socio_encoded,id_prestador_encoded,calificacion_experiencia_encoded,id_zona_encoded,edad_encoded,antiguedad_encoded,sexo_encoded_F,sexo_encoded_M,plan_encoded_a,...,especialidad_prestador_encoded_CLINICA MEDICA,especialidad_prestador_encoded_NUTRICIONISTAS(LIC.),especialidad_prestador_encoded_PEDIATRIA,edad_cat_pediatrico,categoria_prestador_gold,categoria_prestador_silver,categoria_prestador_standard,categoria_socio_gold,categoria_socio_silver,categoria_socio_standard
0,1,100000,200000,0.666667,1,0.952941,0.530769,1,0,1,...,1,0,0,1,1,0,0,0,0,1
1,2,100000,200000,0.555556,1,0.952941,0.530769,1,0,1,...,1,0,0,1,1,0,0,0,0,1
2,4,100002,200000,0.666667,2,0.705882,0.584615,1,0,0,...,1,0,0,1,1,0,0,0,0,1
3,12,100010,200000,0.666667,7,0.505882,0.523077,1,0,1,...,1,0,0,1,1,0,0,0,0,1
4,22,100017,200000,0.666667,4,0.552941,0.538462,1,0,0,...,1,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2001,101548,200193,0.222222,1,0.035294,0.100000,0,1,0,...,0,0,1,1,0,0,1,0,0,1
2496,2080,101612,200193,0.111111,8,0.058824,0.161538,0,1,0,...,0,0,1,1,0,0,1,0,0,1
2497,2088,101618,200194,0.111111,9,0.035294,0.092308,1,0,0,...,0,0,1,1,0,0,1,0,0,1
2498,2327,101791,200195,0.111111,1,0.094118,0.223077,0,1,0,...,0,0,1,1,0,0,1,0,0,1
